# PROJECT 1

## PART 1: Replicating Models from Class

1. Import all libraries

In [1]:
import numpy as np
import pandas as pd
from math import e
from scipy.optimize import minimize
from scipy.stats import gamma
from scipy.stats import poisson
from scipy.stats import gamma
from scipy.optimize import minimize
from math import e
from math import factorial
import math

### Q1: The Poisson Model

#### Consider the example related to billboard exposures from class. The associated data is in the file billboard.csv. Write code to estimate the parameters of the Poisson model using maximum likelihood estimation (MLE).Report your code, the estimated parameters and the maximum value of the log-likelihood.

1. Load the dataset

In [2]:
df = pd.read_csv('billboard.csv')

In [3]:
df.head()

,EXPOSURES,PEOPLE
0,0,48
1,1,37
2,2,30
3,3,24
4,4,20


In [1]:
a = np.array(df[['PEOPLE']]) #define the first array that will go through the PLL function
b = np.array(df[['EXPOSURES']]) #define the second array that will go through PLL
lmbda = 1

def PLL(lmbda,a,b):
    c = 0 #we create a variable that will store the cummulative sum of the iteration below
    for i in range(len(a)): #this will go through through the dataset rows
        c += a[i]*np.log(poisson.pmf(b[i], lmbda)) #we get the sum of PEOPLE[i]*LN(POISSON(EXPOSURES[i],lambda))
    return (-1)*c #we need the non-negative cummulative result

soln = minimize(
    PLL,
    args = (a,b),
    x0 = np.array((1)),
    bounds=[(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
final_lmbda = soln.x[0]
c = 0
for i in range(len(a)):
    c += a[i]*np.log(poisson.pmf(b[i],final_lmbda))

print("Lambda:{}".format(final_lmbda))
print("MAX_LL:{}".format(c))

NameError: name 'np' is not defined

### Q2: The NBD Model

#### Next, write code (for the same dataset) to estimate the parameters of the NBD model using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. Evaluate the NBD model vis-a-vis the Poisson model; explain which is better and why.

In [5]:
df=pd.read_csv('billboard.csv')
df

,EXPOSURES,PEOPLE
0,0,48
1,1,37
2,2,30
3,3,24
4,4,20
5,5,16
6,6,13
7,7,11
8,8,9
9,9,7


In [6]:
def NBD(params,k,t):
    alpha,n = params
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t)))*NBD(params,k-1,t)


def NBDLL(params,t,a,b):
    nbd = 0
    for i in range(len(a)):
        nbd += a[i]*np.log(NBD(params,b[i],t))
    return (-1)*nbd

alpha = 1
n = 1
t = 1
solnbd = minimize(
    NBDLL,
    args = (t,a,b),
    x0 = np.array((1,1)),
    bounds=[(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
alpha = solnbd.x[0]
n = solnbd.x[1]
nbd = 0
params = (alpha,n)
for i in range(len(a)):
    nbd += a[i]*np.log(NBD(params,b[i],t))
print("Alpha:{}".format(alpha))
print("n:{}".format(n))
print("Log likelihood:{}".format(nbd))

Alpha:0.21751778150917153
n:0.9692594289057651
Log likelihood:[-649.68882748]


### Q3: The Poisson Regression

#### Now consider the khakichinos.com example from class; The associated data is in the file khakichinos.csv.Estimate all relevant parameters for Poisson regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood.

In [8]:
df=pd.read_csv('khakichinos.csv')
a=np.array(df[['LnInc']])
b=np.array(df[['Sex']])
c=np.array(df[['LnAge']])
d=np.array(df[['HHSize']])
k=np.array(df[['NumberofVisits']])

def PR(params,a,b,c,d,k):
    lmbda0, beta1, beta2, beta3, beta4 = params
    p = 0
    pt1 = 0
    pt2 = 0
    pt3 = 0
    for i in range(len(a)):
        lmbda= lmbda0*math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])
        fact = float(math.factorial(float(k[i])))
        pt1 = pt1 + k[i]*np.log(lmbda)
        pt2 = pt2 + lmbda
        pt3 = pt3 + np.log(fact)
    p = pt1-pt2-pt3
    return (-1)*p

#calculate exponent 1st
#try not to multiply things together
#take log of 

In [15]:
#CHECKING THAT OUR FUNCTION WORKS
lmbda0=0.0439
beta1=0.0938
beta2=0.0043
beta3=0.5882
beta4=-0.0359
params=(lmbda0,beta1,beta2,beta3,beta4)
test=PR(params,a,b,c,d,k)
test

array([6291.4967508])

In [10]:
lmbda0=1
beta1=1
beta2=1
beta3=1
beta4=1
params=lmbda0,beta1,beta2,beta3,beta4
soln = minimize(
    PR,
    args = (a,b,c,d,k),
    x0 = np.array((1,1,1,1,1)),
    bounds=[(0.00001, None), (None,None), (None,None), (None,None), (None,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

In [20]:
pt1=0
pt2=0
pt3=0

lmbda0 = soln.x[0]
beta1=soln.x[1]
beta2=soln.x[2]
beta3=soln.x[3]
beta4=soln.x[4]
params = (lmbda0, beta1, beta2, beta3, beta4)
for i in range(len(a)):
    lmbda= lmbda0*math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])
    fact = float(math.factorial(float(k[i])))
    pt1 = pt1 + k[i]*np.log(lmbda)
    pt2 = pt2 + lmbda
    pt3 = pt3 + np.log(fact)
p = pt1-pt2-pt3
print("Lambda0: {}".format(lmbda0))
print("Beta1: {}".format(beta1))
print("Beta2: {}".format(beta2))
print("Beta3: {}".format(beta3))
print("Beta4: {}".format(beta4))
print("Log Likelihood: {}".format(p))

Lambda0: 0.04376650692216292
Beta1: 0.09426526605833353
Beta2: 0.004909204441529336
Beta3: 0.5878466719036146
Beta4: -0.03625361512373587
Log Likelihood: [-6291.49715362]


### Q4: The NBD Regression

#### Consider the khakichinos.com example again. Estimate all relevant parameters for NBD Regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood. Evaluate the NBD regression vis-a-vis the Poisson regression; explain which is better and why.

In [12]:
df_k=pd.read_csv('khakichinos.csv')
df_k

,ID,NumberofVisits,LnInc,Sex,LnAge,HHSize
0,1,0,11.379394,1,3.871201,2
1,2,5,9.769956,1,4.043051,1
2,3,0,11.082143,0,3.332205,2
3,4,0,10.915088,1,3.951244,3
4,5,0,10.915088,1,2.833213,3
...,...,...,...,...,...,...
2723,2724,0,9.528794,1,2.944439,2
2724,2725,0,11.379394,0,3.970292,2
2725,2726,0,11.191342,1,3.044522,3
2726,2727,0,10.532096,1,2.890372,4


DEFINING THE EQUATION FOR AN NBD REGRESSION BELOW

In [13]:
df=pd.read_csv('khakichinos.csv')
a=np.array(df[['LnInc']])
b=np.array(df[['Sex']])
c=np.array(df[['LnAge']])
d=np.array(df[['HHSize']])
k=np.array(df[['NumberofVisits']])

def NBDR(params,a,b,c,d,k):
    alpha, n, beta1, beta2, beta3, beta4 = params
    p = 0#we create a variable that will store the cummulative sum of the iteration below
    pt1 = 0
    pt2 = 0
    pt3 = 0
    for i in range(len(a)):
        pt1 = float(math.gamma(n+k[i]))/(float(math.gamma(n))*float(math.factorial(int(k[i]))))
        pt2 = (alpha/(alpha+math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])))**n
        pt3 = (e**(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])/(alpha+float((e**(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])))))**k[i]
        p = p + np.log(pt1*pt2*pt3)
    return (-1)*float(p[0]) #we'll need the negative so we can use 
alpha=8.192937559
n=0.138751195
beta1 =0.07338418
beta2 =-0.009304959
beta3 = 0.90208107
beta4 = -0.02433638
params = (alpha, n, beta1, beta2, beta3, beta4)
NBDR(params,a,b,c,d,k)

2888.9661141906977

In [14]:
alpha=1
n=1
beta1=1
beta2=1
beta3=1
beta4=1
params=(n, beta1, beta2, beta3, beta4, alpha)
solnbdr = minimize(
    NBDR,
    args = (a,b,c,d,k),
    x0 = np.array((1,1,1,1,1,1)),
    bounds=[(0.000001,None),(None,None),(None,None),(None,None),(None,None),(None,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)

# alpha = solnbdr.x[0]
# n = solnbdr.x[1]
# p = 0

# params = (alpha,n,beta1,beta2,beta3)
# for i in range(len(a)): #this will go through through the dataset rows. 
#     pt1 = float(math.gamma(n+k[i]))/(float(math.gamma(n))*float(math.factorial(int(k[i]))))
#     pt2 = (alpha/(alpha+math.exp(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])))**n
#     pt3 = (e**(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])/(alpha+float((e**(beta1*a[i]+beta2*b[i]+beta3*c[i]+beta4*d[i])))))**k[i]
#     p = p+np.log(pt1*pt2*pt3)
# print("Alpha:{}".format(alpha))
# print("n:{}".format(n))
# print("Log likelihood:{}".format(nbd))

<ipython-input-13-8d93c84141b2>:18: RuntimeWarning: invalid value encountered in log
  p = p + np.log(pt1*pt2*pt3)
<ipython-input-13-8d93c84141b2>:18: RuntimeWarning: divide by zero encountered in log
  p = p + np.log(pt1*pt2*pt3)


ZeroDivisionError: float division by zero

In [ ]:
solnbdr

In [ ]:
solnbdr

### Q5: For each of the models above, can you provide some managerial takeaways?

## PART 2: Analysis of New Data

### Q1: Read books.csv and generate two new datasets

In [ ]:
df=pd.read_csv('books.csv')
df

### (a) books01.csv, with the structure of the dataset used in the billboard exposures example (i.e., with only two columns – (i) the number purchases, and (ii) the number of people making the corresponding number of purchases)

In [ ]:
df2 = df['userid']

In [ ]:
df1 = pd.Series(df2.value_counts()).to_frame()

In [ ]:
df3=df1['userid']
df3

In [ ]:
books01 = df3.value_counts().to_frame()
books01['no_of_purch']=books01.index
books01.rename(columns = {'userid':'ppl_purchasing'}, inplace = True)
books01=books01.reset_index(drop=True).sort_values('no_of_purch')
books01

In [ ]:
books01.to_csv('books01.csv', index=False)

In [ ]:
books01=pd.read_csv('books01.csv')
books01

### (b) books02.csv, with the structure of the dataset used in the khakichinos.com example, with a new column containing a count of the number of books purchased from barnesandnoble.com by each customer, while keeping the demographic variables (remember to drop date, product, and price).

In [ ]:
df5=pd.read_csv('books.csv')
# df5=df5[(df5 == 'barnesandnoble.com').any(axis=1)]
df5.loc[df5["domain"].isin(["barnesandnoble.com"]), "domain"]
df5=df5.reset_index(drop=True)
df5

In [ ]:
df6=df5[['userid','qty']]
a=df6.groupby('userid').sum()
a['userid']=a.index #a holds the qty by userid
a.index.names = ['index']
a.head()

In [ ]:
df7=df5.drop(['qty','price','product','date'], axis=1)
b=df7.groupby('userid').mean()
b['userid']=b.index
b.index.names=['index']
b.head()

In [ ]:
books02=pd.merge(a,b,on='userid')
books02.head()

In [ ]:
books02.to_csv('books02.csv', index=False)

In [ ]:
books02=pd.read_csv('books02.csv')
books02.head()

### Print the first and last few records of both new datasets.

### Q2: Develop a Poisson model using books01.csv. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any other information you believe is relevant).

In [ ]:
books01

In [ ]:
df=books01

a = np.array(df[['ppl_purchasing']]) #define the first array that will go through the PLL function
b = np.array(df[['no_of_purch']]) #define the second array that will go through PLL
lmbda = 1

def LL(lmbda,a,b):
    c = 0 #we create a variable that will store the cummulative sum of the iteration below
    for i in range(len(a)): #this will go through through the dataset rows
        c += a[i]*np.log(poisson.pmf(b[i], lmbda)+1) #we get the sum of PEOPLE[i]*LN(POISSON(EXPOSURES[i],lambda))
    return (-1)*c #we need the non-negative cummulative result so that we can maximize ll

soln = minimize(
    LL,
    args = (a,b),
    x0 = np.array((1)),
    bounds=[(0.000001,None)],
    ##tol=1e-10,
    #options={'ftol' : 1e-8},
)
final_lmbda = soln.x[0]
c = 0
for i in range(len(a)):
    c += a[i]*np.log(poisson.pmf(b[i],final_lmbda)+1)

print("Lambda value for poisson model after optimzation {}".format(final_lmbda))
print("Log likelihood function value for poisson model after optimzation {}".format(c))

### Q3: Develop a Poisson model using books02.csv, i.e., by ignoring the independent variables available. Report your code, and confirm that the estimated parameters and the maximum value of the log-likelihood are identical to those obtained with the Poisson model developed using books01.csv.

In [ ]:
new_books=books02.drop(['education','region','hhsz','age','income','child','race','country'],axis=1)
new_books

In [ ]:
# df=books02

# a = np.array(df[['ppl_purchasing']]) #define the first array that will go through the PLL function
# b = np.array(df[['no_of_purch']]) #define the second array that will go through PLL
lmbda = 1

def LL(lmbda,a,b):
    c = 0 #we create a variable that will store the cummulative sum of the iteration below
    for i in range(len(a)): #this will go through through the dataset rows
        c += a[i]*np.log(poisson.pmf(b[i], lmbda)+1) #we get the sum of PEOPLE[i]*LN(POISSON(EXPOSURES[i],lambda))
    return (-1)*c #we need the non-negative cummulative result so that we can maximize ll

soln = minimize(
    LL,
    args = (a,b),
    x0 = np.array((1)),
    bounds=[(0.000001,None)],
    ##tol=1e-10,
    #options={'ftol' : 1e-8},
)
final_lmbda = soln.x[0]
c = 0
for i in range(len(a)):
    c += a[i]*np.log(poisson.pmf(b[i],final_lmbda)+1)

print("Lambda value for poisson model after optimzation {}".format(final_lmbda))
print("Log likelihood function value for poisson model after optimzation {}".format(c))

### Q4: Develop an NBD model using books01.csv. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any other information you believe is relevant).

In [ ]:
books01

In [ ]:
df=pd.read_csv('books01.csv')

def NBD(params,k,t):
    alpha,n = params
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t))) * NBD(params,k-1,t)


def NBDLL(params,t,a,b):
    nbd = 0
    for i in range(len(a)):
        nbd += a[i]*np.log(NBD(params,b[i],t))
    return (-1)*nbd

alpha = 1
n = 1
t = 1
solnbd = minimize(
    NBDLL,
    args = (t,a,b),
    x0 = np.array((1,1)),
    bounds=[(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
alpha = solnbd.x[0]
n = solnbd.x[1]
nbd = 0
params = (alpha,n)
for i in range(len(a)):
    nbd += a[i]*np.log(NBD(params,b[i],t))

print("Alpha:{}".format(alpha))
print("n:{}".format(n))
print("Log likelihood:{}".format(nbd))

### Q5: Develop an NBD model using books02.csv (again, ignoring the variables available). Report your code, and confirm that the estimated parameters and the maximum value of the log-likelihood are identical to those obtained with the NBD model developed using books01.csv.

In [ ]:
df=pd.read_csv('books02.csv')

def NBD(params,k,t):
    alpha,n = params
    if k==0:
        return (alpha/(alpha+t))**n
    else:
        return (((n+k-1)*t)/(k*(alpha+t))) * NBD(params,k-1,t)


def NBDLL(params,t,a,b):
    nbd = 0
    for i in range(len(a)):
        nbd += a[i]*np.log(NBD(params,b[i],t))
    return (-1)*nbd

alpha = 1
n = 1
t = 1
solnbd = minimize(
    NBDLL,
    args = (t,a,b),
    x0 = np.array((1,1)),
    bounds=[(0.000001,None),(0.000001,None)],
    tol=1e-10,
    options={'ftol' : 1e-8},
)
alpha = solnbd.x[0]
n = solnbd.x[1]
t=solnbd.x[]
nbd = 0
params = (alpha,n)
for i in range(len(a)):
    nbd += a[i]*np.log(NBD(params,b[i],t))
print("Alpha:{}".format(alpha))
print("n:{}".format(n))
print('t:{}'.format(t))
print("Log likelihood:{}".format(nbd))

In [ ]:
t

### Q6: Calculate the values of (i) reach, (ii) average frequency, and (iii) gross ratings points (GRPs) based on the NBD model. Show your work.

reach (proportion of population who bought a book at least once at barnesandnobles)

In [ ]:
reach=100*(1-((alpha/(alpha+t))**n))
reach

average frequency (average # of books purchased among those who bought a book)

In [ ]:
avg_frequency=n/alpha/(1-(alpha/(alpha+t)))
avg_frequency

gross ratings points (average # of books purchased per 100 people)

In [ ]:
grps=reach*avg_frequency
grps

### Q7: Identify all independent variables with missing values. How many values are missing in each? Drop any variable with many missing values (specify how you are defining ‘many’). If the number of missing values are very few (again, specify how you are defining ‘few’), delete the rows involved. For the remaining variables (if any), replace the missing values with the means of the corresponding variables. Report your code.

In [ ]:
books02=pd.read_csv('books02.csv')

In [ ]:
print(books02['education'].isna().sum())
print(books02['region'].isna().sum())
print(books02['hhsz'].isna().sum())
print(books02['age'].isna().sum())
print(books02['income'].isna().sum())
print(books02['child'].isna().sum())
print(books02['race'].isna().sum())
print(books02['country'].isna().sum())

In [ ]:
books02=books02.drop(['education'],axis=1)

In [ ]:
avg_region=books02['region'].mean()
avg_age=books02['age'].mean()
print(avg_region)
print(avg_age)

In [ ]:
books02['region']=books02['region'].fillna(avg_region)
books02['age']=books02['age'].fillna(avg_age)

In [ ]:
print(books02['region'].isna().sum())
print(books02['hhsz'].isna().sum())
print(books02['age'].isna().sum())
print(books02['income'].isna().sum())
print(books02['child'].isna().sum())
print(books02['race'].isna().sum())
print(books02['country'].isna().sum())

### Q8: Incorporate the available customer characteristics and estimate all relevant parameters for Poisson regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any other information you believe is relevant). What are the managerial takeaways — which customer characteristics seem to be important?

### Q9: Estimate all relevant parameters for NBD regression using MLE. Report your code, the estimated parameters and the maximum value of the log-likelihood (and any other information you believe is relevant). What are the managerial takeaways — which customer characteristics seem to be important?

### Q10: Evaluate all the models developed using the log-likelihood ratio, AIC, and BIC. What are your recommendations on which model to use based on each of these criteria? Are the recommendations consistent? Explain why you are recommending the model you have selected. Are there any significant differences among the results from the models? If so, what exactly are these differences? Discuss what you believe could be causing the differences.

### Q11: Briefly summarize what you learned from this project. This is an open-ended question, so please include anything you found worthwhile — relating to the modeling process, insights from the process and models, any managerial takeaways that were insightful to you, and so on.